### Setup

In [2]:
import pandas as pd
import ast
import os
from dotenv import load_dotenv
from sqlalchemy import (create_engine, Table, Column, Integer, String, MetaData, insert, VARCHAR)

In [ ]:
local = pd.read_csv('datasets/crawler/local_codes.csv')
foreign = pd.read_csv('datasets/crawler/foreign_codes.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

In [3]:
load_dotenv()
user = os.getenv('USER')
password = os.getenv('PW')
eng = (
    create_engine(
        'mysql+pymysql://' + user + ':' + password + '@localhost/legislativas', 
        pool_recycle=3600, echo=True)
)
meta = MetaData()

### Methods

In [ ]:
# Returns resuts table df
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(
            ['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], 
            axis=1)
        .rename(columns={'acronym': 'party'})
        .set_index('party')
    )

    return results

# Returns metadata df
def getResults(file, row):
    md = pd.Series(file['currentResults'])
    md.index.name = None
    md = (
        md.drop(
            ['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty', 
            'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts',
            'totalMandates', 'totalParishesApproved'])
        .to_frame()
        .T
        .assign(territoryKey = row['territoryKey'])
        .set_index('territoryKey')
    )
    
    loc = (
        pd.DataFrame(
            [row['parish'], row['county'], row['district'], row['territoryKey']]
            , index=['parish', 'county', 'district', 'territoryKey'])
        .T
        .set_index('territoryKey')
    )
    
    r = getResultsTable(file).drop(['percentage', 'validVotesPercentage'], axis=1).T
    r['territoryKey'] = row['territoryKey']
    r = r.reset_index().set_index('territoryKey').drop(['index'], axis=1)

    return r, md, loc

# Creates directories
def mkdir():
        path = '/datasets/results/' 

        working_dir = os.getcwd()
        newpath = working_dir + path

        if not os.path.exists(newpath):
            os.makedirs(newpath)

In [4]:
def load_files():
    results = pd.read_csv('datasets/results/results.csv').set_index('territoryKey')
    metadata = pd.read_csv('datasets/results/metadata.csv').set_index('territoryKey')
    location = pd.read_csv('datasets/results/location.csv').set_index('territoryKey')

    return results, metadata, location

def get_strmax(results, location):
    lcol, strmax = location.columns.tolist(), pd.Series()
    strmax['territoryKey'] = results.index.get_level_values('territoryKey').str.len().max()
    
    for col in lcol:
        if location[col].dtype == 'object':
            strmax[col] = location[col].str.len().max()
            
    return strmax

### Create Results files

In [ ]:
results = pd.DataFrame()
metadata = pd.DataFrame()
location = pd.DataFrame()

for i, row in codes.iterrows():
    file = (
        pd.read_csv('datasets/crawler/' + row['territoryKey'].split('-')[0].capitalize() + '/' + row['district'] + '/' + row['county'] + '/' + row['parish'] + '.csv')
        .set_index('index')
    )
    
    r, md, loc = getResults(file, row)

    results = pd.concat([results, r])    
    metadata = pd.concat([metadata, md])
    location = pd.concat([location, loc])

In [ ]:
mkdir()

results.to_csv('datasets/results/results.csv')
metadata.to_csv('datasets/results/metadata.csv')
location.to_csv('datasets/results/location.csv')

### Database

#### Table creation

In [6]:
results, metadata, location = load_files()
strmax = get_strmax(results, location)

with eng.connect() as con:
    resultsTable = Table('results', meta,
        Column('territoryKey', String(strmax['territoryKey']), primary_key=True, nullable=False),
        Column('PPD/PSD.CDS-PP.PPM', Integer),
        Column('PS', Integer),
        Column('CH', Integer),
        Column('B.E.', Integer),
        Column('IL', Integer),
        Column('ADN', Integer),
        Column('L', Integer),
        Column('PAN', Integer),
        Column('PCP-PEV', Integer),
        Column('VP', Integer),
        Column('E', Integer),
        Column('JPP', Integer),
        Column('R.I.R.', Integer),
        Column('ND', Integer),
        Column('PCTP/MRPP', Integer),
        Column('MPT.A', Integer),
        Column('NC', Integer),
        Column('PTP', Integer),
        Column('PPD/PSD.CDS-PP', Integer),
        Column('PPM', Integer),        
    )
    metadataTable = Table('metadata', meta,
        Column('territoryKey', String(strmax['territoryKey']), primary_key=True, nullable=False),
        Column('blankVotes', Integer),
        Column('blankVotesPercentage', Integer),
        Column('nullVotes', Integer),
        Column('nullVotesPercentage', Integer),
        Column('numberParishes', Integer),
        Column('numberVoters', Integer),
        Column('percentageVoters', Integer),
        Column('subscribedVoters', Integer),
        Column('totalVoters', Integer)
    )
    locationTable = Table('location', meta,
        Column('territoryKey', String(strmax['territoryKey']), primary_key=True, nullable=False),
        Column('parish', String(strmax['parish'])),
        Column('county', String(strmax['county'])),
        Column('district', String(strmax['district']))
    )
    
    meta.create_all(eng)
    # meta.drop_all(eng)
    # meta.reflect(bind=eng) # Connect to database

2024-04-23 17:11:05,687 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-23 17:11:05,688 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,690 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-23 17:11:05,690 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,691 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-23 17:11:05,692 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,710 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-23 17:11:05,710 INFO sqlalchemy.engine.Engine DESCRIBE `legislativas`.`results`
2024-04-23 17:11:05,710 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,714 INFO sqlalchemy.engine.Engine DESCRIBE `legislativas`.`metadata`
2024-04-23 17:11:05,714 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,714 INFO sqlalchemy.engine.Engine DESCRIBE `legislativas`.`location`
2024-04-23 17:11:05,714 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:11:05,714 INFO sq

#### Import from df to mySQL

In [8]:
results.to_sql(
    'results',
    con=eng,
    if_exists='replace',
    dtype={
        'territoryKey': VARCHAR(strmax['territoryKey'])
    }
)
metadata.to_sql(
    'metadata',
    con=eng,
    if_exists='replace',
    dtype={
        'territoryKey': VARCHAR(strmax['territoryKey'])
    }
)
location.to_sql(
    'location',
    con=eng,
    if_exists='replace',
    dtype={
        'territoryKey': VARCHAR(strmax['territoryKey'])
    }
)

2024-04-23 17:12:14,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine DESCRIBE `legislativas`.`results`
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine DESCRIBE `legislativas`.`results`
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `legislativas`
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `legislativas`
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `results`
2024-04-23 17:12:14,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-23 17:12:14,239 INFO sqlalchemy.engine.Engine 
DROP TABLE results
2024-04-23 17:12:14,239 INFO sqlalchemy.engine.Engine [no key 0.00054s] {}
2024-

3116

### Explore Data

In [ ]:
# results, metadata, location = load_files()

# parties = results.index.unique(level='party')

# idx = pd.IndexSlice

In [8]:
meta.reflect(bind=eng)



2024-04-24 10:58:04,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-24 10:58:04,996 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `legislativas`
2024-04-24 10:58:04,996 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-24 10:58:04,999 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `location`
2024-04-24 10:58:04,999 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-24 10:58:05,001 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `metadata`
2024-04-24 10:58:05,002 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-24 10:58:05,003 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `results`
2024-04-24 10:58:05,004 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-24 10:58:05,006 INFO sqlalchemy.engine.Engine ROLLBACK


AttributeError: 'MetaData' object has no attribute 'location'